In [1]:
import pywikibot
import pandas as pd
from tqdm.notebook import tqdm
import time
import numpy as np

In [2]:
df = pd.read_excel("Tabel van verkiezingen.xlsx")
# ignore rows with empty values in references of election
df = df.dropna(how='any',subset=['source_url','retrieved_date', 'Election_WD_id','Person_WD_id'])
df.tail()

,List,Party,Candidate number,Candidate name,N_votes,Election,Party_WD_id,Person_WD_id,Election_WD_id,source_url,retrieved_date
1052,24.0,Lokaal in de Kamer,24,"de Bree, C.M.",116,Tweede Kamer 2017,NaN,Q28862324,Q16061881,https://www.kiesraad.nl/actueel/nieuws/2017/03...,2020-12-28
1053,24.0,Lokaal in de Kamer,25,"Barends, R.",122,Tweede Kamer 2017,NaN,Q28862325,Q16061881,https://www.kiesraad.nl/actueel/nieuws/2017/03...,2020-12-28
1054,24.0,Lokaal in de Kamer,26,"Dirkmaat, G.B.",32,Tweede Kamer 2017,NaN,Q28862327,Q16061881,https://www.kiesraad.nl/actueel/nieuws/2017/03...,2020-12-28
1055,24.0,Lokaal in de Kamer,27,"Kloos, C.H.",197,Tweede Kamer 2017,NaN,Q28862329,Q16061881,https://www.kiesraad.nl/actueel/nieuws/2017/03...,2020-12-28
1056,25.0,JEZUS LEEFT,1,"van der Spek, F.I.A.",2611,Tweede Kamer 2017,Q28660340,Q66725360,Q16061881,https://www.kiesraad.nl/actueel/nieuws/2017/03...,2020-12-28


In [ ]:
site = pywikibot.Site("wikidata")
repo = site.data_repository()

candidacy = 'P3602'

for index, row in tqdm(df.iterrows()):
    if index > 2:
        break
    add_reference = False
    item = pywikibot.ItemPage(repo, row["Person_WD_id"])
    item.get()
    if candidacy in item.claims:
        list_of_claims = [
            claim for claim in item.claims[candidacy]
            if row['Election_WD_id'] in str(claim.getTarget())
        ]
    else:
        list_of_claims = []

    if len(list_of_claims) == 0:
        claim = pywikibot.Claim(repo, candidacy)
        claim.setTarget(pywikibot.ItemPage(repo, row['Election_WD_id']))
        item.addClaim(claim,
                      summary=u'Added candidacy in %s' % row['Election_WD_id'])
        add_reference = True
        # Create new list of claims
        list_of_claims = [
            claim for claim in item.claims[candidacy]
            if row['Election_WD_id'] in str(claim.getTarget())
        ]

    # Add position on list
    if 'P1545' not in item.claims[candidacy][0].qualifiers and not pd.isna(row["Candidate number"]):
        qualifier = pywikibot.Claim(repo, u'P1545')
        qualifier.setTarget(pywikibot.WbQuantity(row["Candidate number"]))
        item.claims[candidacy][0].addQualifier(
            qualifier, summary=u"Added position on list")
        add_reference = True

    # Add party
    if 'P1268' not in item.claims[candidacy][0].qualifiers and not pd.isna(row['Party_WD_id']):
        qualifier = pywikibot.Claim(repo, u'P1268')
        qualifier.setTarget(pywikibot.ItemPage(repo, row['Party_WD_id']))
        item.claims[candidacy][0].addQualifier(
            qualifier, summary=u"Added represented party")
        add_reference = True

    # Add number of votes
    if 'P1111' not in item.claims[candidacy][0].qualifiers and not pd.isna(row["N_votes"]):
        qualifier = pywikibot.Claim(repo, u'P1111')
        qualifier.setTarget(pywikibot.WbQuantity(row["N_votes"]))
        item.claims[candidacy][0].addQualifier(
            qualifier, summary=u"Added number of votes")
        add_reference = True

    if add_reference:
        url = pywikibot.Claim(repo, u'P854')
        url.setTarget(row["source_url"])

        retrieved = pywikibot.Claim(repo, u'P813')
        date = pywikibot.WbTime(year=2020, month=12, day=28)
        retrieved.setTarget(date)

        item.claims[candidacy][0].addSources([url, retrieved],
                                             summary=u'Adding sources.')
    else:
        continue
    time.sleep(8)